# To do list

[Weather data](#Weather-data)
- [x] 2m_temperature daily => t_air_24,t_air_min_24, t_air_max_24
- [x] 2m_dewpoint_temperature daily => vp_24
- [x] 10m_wind_speed daily => u_24
- 2m_relative_humidity (available, not used)
- vapour_pressure (available, not used)

[Solar radiation](#Solar-radiation)
- [x] solar_radiation_flux daily => ra_24

In [7]:
%run ..\Functions.ipynb

# Test

In [17]:
datestr='20190101'
folder=r"E:\WaPORQA\AgERA5"
day_fhs=glob.glob(os.path.join(folder,'2m_temperature',
                               '*{0}*.nc'.format(datestr))) + \
    glob.glob(os.path.join(folder,'2m_dewpoint_temperature',
                                   '*{0}*.nc'.format(datestr))) + \
    glob.glob(os.path.join(folder,'10m_wind_speed',
                                   '*{0}*.nc'.format(datestr))) + \
    glob.glob(os.path.join(folder,'solar_radiation_flux',
                                   '*{0}*.nc'.format(datestr))) + \
    glob.glob(os.path.join(folder,'2m_relative_humidity',
                                   '*{0}*.nc'.format(datestr)))
day_fhs

['E:\\WaPORQA\\AgERA5\\2m_temperature\\23e7e4d2-80a3-46e2-8539-c59980aa1e8f-Temperature-Air-2m-Max-24h_C3S-glob-agric_AgERA5_20190101_final-v1_area_subset.nc',
 'E:\\WaPORQA\\AgERA5\\2m_temperature\\23e7e4d2-80a3-46e2-8539-c59980aa1e8f-Temperature-Air-2m-Mean-24h_C3S-glob-agric_AgERA5_20190101_final-v1_area_subset.nc',
 'E:\\WaPORQA\\AgERA5\\2m_temperature\\23e7e4d2-80a3-46e2-8539-c59980aa1e8f-Temperature-Air-2m-Min-24h_C3S-glob-agric_AgERA5_20190101_final-v1_area_subset.nc',
 'E:\\WaPORQA\\AgERA5\\2m_dewpoint_temperature\\c211d578-ae40-4dc5-8297-ca95aff9f5f9-Dew-Point-Temperature-2m-Mean_C3S-glob-agric_AgERA5_20190101_final-v1_area_subset.nc',
 'E:\\WaPORQA\\AgERA5\\10m_wind_speed\\3d5c9425-1ee6-41c0-a73d-5674a36eeb1e-Wind-Speed-10m-Mean_C3S-glob-agric_AgERA5_20190101_final-v1_area_subset.nc',
 'E:\\WaPORQA\\AgERA5\\solar_radiation_flux\\19950260-5b7b-46a6-b79b-821fef7fa129-Solar-Radiation-Flux_C3S-glob-agric_AgERA5_20190101_final-v1_area_subset.nc',
 'E:\\WaPORQA\\AgERA5\\2m_relative

In [18]:
ds=xr.open_mfdataset(day_fhs)
ds

<xarray.Dataset>
Dimensions:                        (lat: 800, lon: 950, time: 1)
Coordinates:
  * lat                            (lat) float64 39.9 39.8 39.7 ... -39.9 -40.0
  * lon                            (lon) float64 -29.9 -29.8 -29.7 ... 64.9 65.0
  * time                           (time) datetime64[ns] 2019-01-01
Data variables:
    Dew_Point_Temperature_2m_Mean  (time, lat, lon) float32 dask.array<chunksize=(1, 800, 950), meta=np.ndarray>
    Relative_Humidity_2m_06h       (time, lat, lon) float32 dask.array<chunksize=(1, 800, 950), meta=np.ndarray>
    Relative_Humidity_2m_09h       (time, lat, lon) float32 dask.array<chunksize=(1, 800, 950), meta=np.ndarray>
    Relative_Humidity_2m_12h       (time, lat, lon) float32 dask.array<chunksize=(1, 800, 950), meta=np.ndarray>
    Relative_Humidity_2m_15h       (time, lat, lon) float32 dask.array<chunksize=(1, 800, 950), meta=np.ndarray>
    Relative_Humidity_2m_18h       (time, lat, lon) float32 dask.array<chunksize=(1, 800, 950), meta=np.ndarray>
    Solar_Radiation_Flux           (time, lat, lon) float32 dask.array<chunksize=(1, 800, 950), meta=np.ndarray>
    Temperature_Air_2m_Max_24h     (time, lat, lon) float32 dask.array<chunksize=(1, 800, 950), meta=np.ndarray>
    Temperature_Air_2m_Mean_24h    (time, lat, lon) float32 dask.array<chunksize=(1, 800, 950), meta=np.ndarray>
    Temperature_Air_2m_Min_24h     (time, lat, lon) float32 dask.array<chunksize=(1, 800, 950), meta=np.ndarray>
    Wind_Speed_10m_Mean            (time, lat, lon) float32 dask.array<chunksize=(1, 800, 950), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.7

## Calculate intermediate variables

In [14]:
t_dew = kelvin_to_celsius(ds.Dew_Point_Temperature_2m_Mean, 
                          attrs={'input': 'AgERA5 Dew_Point_Temperature_2m_Mean'})
save_ds = xr.Dataset({'t_air': kelvin_to_celsius(ds.Temperature_Air_2m_Mean_24h,
                                                attrs={'input':'AgERA5 Temperature_Air_2m_Mean_24h'}
                                                ), 
                      't_air_min': kelvin_to_celsius(ds.Temperature_Air_2m_Min_24h,
                                                attrs={'input':'AgERA5 Temperature_Air_2m_Min_24h'}
                                                ), 
                      't_air_max': kelvin_to_celsius(ds.Temperature_Air_2m_Max_24h,
                                                attrs={'input':'AgERA5 Temperature_Air_2m_Max_24h'}
                                                ), 
                     'u': ds.Wind_Speed_10m_Mean,
                     'vp': vapour_pressure_from_dewpoint(t_dew,
                                                    attrs={'input':'AgERA5 Dew_Point_Temperature_2m_Mean'}     
                                                                 ),
                      # 'ra': ds
                     })

# Preprocessing all files

In [8]:
# import the datetime module
import datetime

folder=r"E:\WaPORQA\AgERA5"
save_folder=r"E:\WaPORQA\AgERA5\Processed"
# create a range of dates from 2018-01-01 to 2022-12-31
start_date = datetime.date(2018, 1, 1)
end_date = datetime.date(2022, 12, 31)
date_range = [start_date + datetime.timedelta(days=x) for x in range((end_date - start_date).days + 1)]

# for each date print a string with this format YYYYMMDD
for date in date_range:
    datestr=date.strftime("%Y%m%d")
    day_fhs=glob.glob(os.path.join(folder,'2m_temperature',
                                   '*{0}*.nc'.format(datestr))) + \
        glob.glob(os.path.join(folder,'2m_dewpoint_temperature',
                                       '*{0}*.nc'.format(datestr))) + \
        glob.glob(os.path.join(folder,'10m_wind_speed',
                                       '*{0}*.nc'.format(datestr))) + \
        glob.glob(os.path.join(folder,'solar_radiation_flux',
                                       '*{0}*.nc'.format(datestr))) + \
        glob.glob(os.path.join(folder,'2m_relative_humidity',
                                       '*{0}*.nc'.format(datestr)))
    ds=xr.open_mfdataset(day_fhs)
    t_dew = kelvin_to_celsius(ds.Dew_Point_Temperature_2m_Mean, 
                          attrs={'input': 'AgERA5 Dew_Point_Temperature_2m_Mean'})
    ra_w = J_per_day_to_W(ds.Solar_Radiation_Flux, 
                          attrs={'input': 'AgERA5 Solar_Radiation_Flux'})
    save_ds = xr.Dataset({'t_air': kelvin_to_celsius(ds.Temperature_Air_2m_Mean_24h,
                                                    attrs={'input':'AgERA5 Temperature_Air_2m_Mean_24h'}
                                                    ), 
                          't_air_min': kelvin_to_celsius(ds.Temperature_Air_2m_Min_24h,
                                                    attrs={'input':'AgERA5 Temperature_Air_2m_Min_24h'}
                                                    ), 
                          't_air_max': kelvin_to_celsius(ds.Temperature_Air_2m_Max_24h,
                                                    attrs={'input':'AgERA5 Temperature_Air_2m_Max_24h'}
                                                    ), 
                         'u': ds.Wind_Speed_10m_Mean,
                         'vp': vapour_pressure_from_dewpoint(t_dew,
                                                        attrs={'input':'AgERA5 Dew_Point_Temperature_2m_Mean'}     
                                                                     ),
                          'ra': ra_w
                         })
    results = save_ds.compute()
    datename=date.strftime("%Y-%m-%d")
    filepath=os.path.join(save_folder,datename+"_t-u-vp-ra.nc")
    results.to_netcdf(filepath)
                          
